<a href="https://colab.research.google.com/github/StanciuAlin/Crypto-Trend-DeepLearning/blob/main/Crypto_Trend_Prediction_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf

# Setăm seed-ul pentru reproductibilitate (cerință metodologie)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Descărcăm prețurile zilnice pentru Bitcoin
df = yf.download('BTC-USD', start='2020-01-01', end='2026-01-01')
# Păstrăm doar prețul de închidere (univariate baseline)
data = df.filter(['Close'])
dataset = data.values

In [ ]:
# Normalizăm datele între 0 și 1 [cite: 50]
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Definim fereastra de timp (look-back): 60 de zile [cite: 207]
prediction_days = 60
X_train, y_train = [], []

for i in range(prediction_days, len(scaled_data)):
    X_train.append(scaled_data[i-prediction_days:i, 0])
    y_train.append(scaled_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2), # Prevenire overfitting [cite: 124]
    LSTM(units=50, return_sequences=False),
    Dropout(0.2),
    Dense(units=25),
    Dense(units=1) # Predicția prețului următor
])

model.compile(optimizer='adam', loss='mean_squared_error')
# Antrenăm modelul
model.fit(X_train, y_train, batch_size=32, epochs=20)